In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Data Sets

In [ ]:
# 데이터 처리을 위한 Pandas 및 Numpy
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np


# 시각화를 위한 matplotlib 및 seaborn
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


# pandas에서 나오는 경고문 무시
# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

# EDA

## Note : 모든 대출에는 자체 행이 있으며 SK_ID_CURR로 식별함
* target == 0 : 대출금 상환
* target == 1 : 대출금 상환 x

In [ ]:
# Training data
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)

# Train data 확인
app_train.head()

In [ ]:
# Testing data 확인
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

### train 데이터의 tartget 비율 확인

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
# 히스토그램으로 보기
app_train['TARGET'].astype(int).plot.hist()

0 클래스가 많음 
* 기간 내에 갚은 대출의 비율이 많음
* 편향을 반영하기 위해 클래스에 weight을 부여해야 함

### 강한 상관관계를 갖는 변수 살펴보기
  
#### EXT_SOURCE 변수

In [ ]:
# Target과 EXT_SOURCE 변수들 corr 확인
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize = (8, 6))

# 히트맵으로 시각화
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

* EXT_SOURCE 3개 모두 target과 음의 관계를 보이고 있음
    * 음의 관계가 커질 수록 제대로 갚을 가능성이 높음

* EXT_SOURCE_3의 상관 관계가 제일 큼

In [ ]:
# 각 변수가 target에 주는 영향 시각화
plt.figure(figsize = (10, 12))

# 3개 변수에 모두 진행
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    
    plt.subplot(3, 1, i + 1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

히트맵과 마찬가지로 3번이 가장 상환과 관계가 있음을 알 수 있음

# Feature Engineering

## FE 1 : 고객들의 이전 대출 파악
* groupby : SK_ID_CURR 의 값에 따라 고객별로 그룹화
* agg : 최소, 최대, 평균, 합계 등을 계산
* merge : 집계된 데이터를 원본 train 데티터와 merge하고 없을 경우 Nan으로 처리

In [ ]:
# Read the bureau file
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
# 확인
bureau.head()

In [ ]:
# SK_ID_CURR을 기준으로 groupby하여 총합을 구하고 동시에 Col name을 바꿈
previous_loan_counts=bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

## FE 2 : train data와 merge

In [ ]:
# train data 읽기
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
# merge
train = train.merge(previous_loan_counts, on='SK_ID_CURR', how = 'left')

In [ ]:
# missing value 처리 -> nan 값은 0으로 처리
train['previous_loan_counts'] = train['previous_loan_counts'].fillna(0)
train.head()

previous_loan_counts가 생성된 것을 확인할 수 있음

## KDE 그래프
### 생성한 column의 유용성 확인
* target 대한 r-value가 커질 수록, 생성한 column이 유용할 가능성이 높아지므로 가장 큰 r-value를 갖는 변수를 찾아야 함

In [ ]:
# 다양한 변수에 이용할 수 있도록 함수로 작성
def kde_target(var_name, df):
    
    # 새롭게 생성된 column과 target 사이의 상관계산
    corr = df['TARGET'].corr(df[var_name])
    
    # 제대로 갚은 그룹과 갚지 못한 그룹 각각의 median
    avg_repaid = df.loc[df['TARGET']==0,var_name].median()
    avg_not_repaid = df.loc[df['TARGET']==1,var_name].median()
    
    plt.figure(figsize = (12,6))
    
    # target에 따라 달라지는 색상
    sns.kdeplot(df.loc[df['TARGET']==0,var_name],label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET']==1,var_name],label = 'TARGET == 1')
    
    # 그래프 라벨링
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # 상관계수와 중앙값 print
    print('[%s]와 타겟 사이의 corr : %0.4f' % (var_name, corr))
    print('제대로 갚지 못한 사람 중앙값 : %0.4f' % avg_not_repaid)
    print('제대로 갚은 사람 중앙값 :  %0.4f' % avg_repaid)

가장 중요한 변수로 밝혀진 EXT_SOURCE_3 테스트

In [ ]:
kde_target('EXT_SOURCE_3', train)

위에서 만든 previous_loan_counts

In [ ]:
kde_target('previous_loan_counts', train)

상관계수가 작고 target 값과 분포 차이가 거의 없기 때문에 previous_loan_counts는 중요하지 않은 변수임을 알 수 있음

## FE 3 : AGGREGATE - 수치 데이터 대표값 계산

bureau 데이터 프레임 안의 수치 데이터를 활용할 수 있도록 모든 수치 데이터의 집계값들을 계산한다. 
* 고객 별로 groupby를 하고
* groupby된 column 들을 집계하고
* 결과를 train dataset과 merge한다

In [ ]:
# 고객 별로 groupby하고 agg 연산 수행
bureau_agg = bureau.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR',as_index=False).agg(['count','mean','max','min','sum']).reset_index()
# 확인
bureau_agg.head()

## FE 4 : multi-level index를 single-level 로 변환

In [ ]:
# 컬럼 리스트 생성
columns = ['SK_ID_CURR']

# 반복문 실행
for var in bureau_agg.columns.levels[0]:
    if var != 'SK_ID_CURR':
        
        # 대표값의 종류에 따라 반복문 생성
        for stat in bureau_agg.columns.levels[1][:-1]:
            # 변수와 대표값의 종류에 따라 새로운 column name 생성
            columns.append('bureau_%s_%s' % (var, stat))


In [ ]:
# 생성된 리스트를 데이터프레임의 column name으로 설정
bureau_agg.columns = columns
# 확인
bureau_agg.head()

In [ ]:
# train data와 merge
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
train.head()

In [ ]:
# target과 위의 대표값들의 corr 분석
new_corrs = []

for col in columns:
    # target과의 상관계수 계산
    corr = train['TARGET'].corr(train[col])
    
    # 리스트에 추가
    new_corrs.append((col, corr))

In [ ]:
# 상관계수가 큰 순서대로 나열
new_corrs = sorted(new_corrs, key = lambda x: abs(x[1]), reverse = True)

In [ ]:
# corr 계산까지 완료한 colum들 확인
new_corrs[:15]

In [ ]:
# 확인
kde_target('bureau_DAYS_CREDIT_mean', train)

대출신청까지 걸린 기간에 대한 그래프다. 
이전에 타 기관에서 대출을 받고 현재 대출을 받기까지 걸린 일수를 의미하기 때문에 음수값이 크다는 것은 타 기관에서 대출을 받은 시점이 오래 된 것을 뜻한다. 즉, 약한 상관관계를 갖는다면 오래 전에 대출을 신청한 사람들은 제대로 갚을 확률이 높다는 것을 의미한다

## FE 5 : 수치 데이터의 대표값 구하기

In [ ]:
def agg_numeric(df, group_var, df_name):
    '''
    데이터프레임 안의 수치형 데이터들의 대표값을 계산
    df : 연산의 대상
    group_var : groupby 되는 col
    df_name : 새로운 col 이름
    '''
    
    # groupby가 되지 않는 col 지우기
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns =col)
        
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids
    
    # 특정 변수 그룹화하고 대표값 계싼
    agg = numeric_df.groupby(group_var).agg(['count','mean','max','min','sum']).reset_index()
        
    # 새로운 col 이름 만들기
    columns = [group_var]
    
    for var in agg.columns.levels[0]:
        # id col 생략
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s' % (df_name, var, stat))
        
    agg.columns = columns
    return agg

In [ ]:
# 생성된 함수 확인
bureau_agg.head()

In [ ]:
# target과 확인하고자 하는 변수 사이의 corr 계산하는 함수

def target_corrs(df):
    corrs = []
    for col in df.columns:
        # target column은 제외
        if col != 'TARGET':
            # target과의 corr계산
            corr = df['TARGET'].corr(df[col])
            corrs.append((col, corr))
    
    # 상관계수가 큰 순서대로 정렬
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    return corrs

## FE 6 : 범주형 데이터 처리 - Encoding

In [ ]:
# 범주형 컬럼들에 대해 one-hot 인코딩 적용
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

In [ ]:
# sum : 총 대출 회수
# mean : 회수의 평균 - 정규화 시킨 수치
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

multi-level 인덱스로 작성되어 있는 column들을 처리해야 함  
level0이 범주형 데이터의 column 이름이기 때문에 level0을 iterator로 활용하여 전체 범주형 데이터에 대해 반복문을 돌릴 것이고, 그 안에서 계산된 통계에 대해 이중 반복문을 사용한다. 
* level 0 : CREDIT_ACTIVE_Activate 같은 column 이름
* level 1 : sum이나 mean 같은 통계 이름

In [ ]:
print(categorical_grouped.columns.levels[0][:5])
print('-----'*10)
print(categorical_grouped.columns.levels[1])

In [ ]:
group_var = 'SK_ID_CURR'

# 새로운 column 제목들을 저장하기 위한 리스트를 생성
columns = []

# 변수(원본 데이터프레임의 column name)에 따라 반복문을 실행
for var in categorical_grouped.columns.levels[0]:
    
    # 고객 id column은 생략
    if var != group_var:
        
        # 통계치의 종류에 따라 반복문을 실행
        for stat in ['count','count_norm']:
            
            # 새로운 column 제목을 정의
             columns.append('%s_%s' % (var, stat))
                
# column들을 재정의
categorical_grouped.columns = columns

categorical_grouped.head()

* sum column : 총 회수
* count_norm column : 위의 회수를 정규화 시킨 것

In [ ]:
# 훈련 데이터와 합치기
train = train.merge(categorical_grouped, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
train.head()

In [ ]:
train.shape

## FE 7 : 범주형 데이터를 처리하기 위한 함수
범주화 데이터에 대해서도 counts와 정규화된 counts를 계산

In [ ]:
def count_categorical(df, group_var, df_name):
    # 범주형 데이터 column
    categorical = pd.get_dummies(df.select_dtypes('object'))
    # column 이름 넣어주기
    categorical[group_var] = df[group_var]
    # group_var의 column을 기준으로 groupby하고 sum과 mean 연산
    categorical = categorical.groupby(group_var).agg(['sum','mean'])
    
    column_names = []
    
    # levels[0]에는 컬럼의 이름이 저장되어 있음
    for var in categorical.columns.levels[0]:
        # levels[1]에는 통계가 저장되어 있음
         for stat in ['count','count_norm']:
                column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    return categorical

In [ ]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

### bureau balance 데이터에도 적용

In [ ]:
# 'bureau blance' 데이터
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
# 확인
bureau_balance.head()

In [ ]:
# 이전 대출에 대해 status 개수 파악
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

In [ ]:
# 각각의 SK_ID_CURR 별 대표값들을 계산
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')

bureau_balance_agg.head()

In [ ]:
# loan 기준으로 데이터프레임 groupby
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

In [ ]:
# SK_ID_CURR을 포함하여 병합
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')
bureau_by_loan.head()

In [ ]:
# 각 고객별로 수치형 대표값들을 계산
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
bureau_balance_by_client.head()

## FE 8 : 생성된 정보들 병함

In [ ]:
# 가공 전 원본 데이터들 가져오기
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')

In [ ]:
# bureau 데이터 내의 범주형 데이터 개수 
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

In [ ]:
# bureau 데이터의 대표값 계산
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

In [ ]:
# bureau balance의 각 대출별 범주형 데이터 개수

bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

In [ ]:
# bureau balance의 각 대출별 대표값 계산
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

In [ ]:
# bureau balance 데이터의 고객별 대표값 계산

## 대출별로 데이터 merge
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

## SK_ID_CURR 포함
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

## 고객별로 대표값 계산
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')

## FE 9 : 위의 feature를 train과 merge

In [ ]:
# 원본 피쳐
original_features = list(train.columns)
print('원본 피쳐 개수 ', len(original_features))

In [ ]:
# bureau_count Merge
train = train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')

# bureau_agg Merge
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

# 월별 고객 정보와 Merge
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

## FE 10: Missing value 처리


In [ ]:
def missing_values_table(df):
    
    # null 값 총계
    mis_val = df.isnull().sum()
    # null 값 비율
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    # null 값의 결과 표 작성
    mis_val_table = pd.concat([mis_val,mis_val_percent], axis=1)
    # column 이름 설정
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

In [ ]:
missing_train = missing_values_table(train)
missing_train.head(10)

In [ ]:
# 누락 비율이 80% 이상 누락된 것 추출
missing_train_vars = list(missing_train.index[missing_train['% of Total Values'] > 80])


In [ ]:
# 테스트에서도 80% 이상 제거하기 위해 read
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

# bureau counts와 merge
test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
# bureau balance와 merge
test = test.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

## FE 11 : train 과 test column 맞춰주기
one-hot 인코딩을 할 때, 두개의 데이터 프레임이 동일한 column을 갖고 있어야 함

In [ ]:
train_labels = train['TARGET']

train, test = train.align(test, join = 'inner', axis = 1)

train['TARGET'] = train_labels

In [ ]:
# test data에서도 80% 이상 누락된 것 추출
missing_test = missing_values_table(test)
missing_test_vars = list(missing_test.index[missing_test['% of Total Values'] > 80])

# test & train 80% 이상 누락 리스트 
missing_columns = list(set(missing_test_vars + missing_train_vars))

In [ ]:
# 80% 이상 누락 column drop
train = train.drop(columns = missing_columns)
test = test.drop(columns = missing_columns)

## FE 12 : 기존의 결과보다 더 높은 corr 찾기

In [ ]:
# 데이터 프레임 안의 모든 상관계수 계산
corrs = train.corr()

In [ ]:
corrs = corrs.sort_values('TARGET', ascending = False)
# top 10 확인
print(pd.DataFrame(corrs['TARGET'].head(10)))
print('-----'*10)
print(pd.DataFrame(corrs['TARGET'].dropna().tail(10)))

In [ ]:
# KDE plot으로 얼마나 유용한지 확인
kde_target(var_name='client_bureau_balance_MONTHS_BALANCE_count_mean', df=train)

각 고객의 대출에 대해 월별 기록의 평균을 나타낸 그래프다.
분산 그래프에 의해, 과거에 월별 평균이 높은 사람들이 제대로 갚는 것을 볼 수 있다.
더 높은 신용 기록을 가지는 고객들이 제대로 갚을 가능성이 높음을 확인했다.

### target을 제외한 변수들간의 상관계수가 0.75 이상인 것들만 저장

In [ ]:
# 상관관계가 높은 변수들 저장하는 딕셔너리
above = {}

for col in corrs:
    above[col] = list(corrs.index[corrs[col]>0.75])

In [ ]:
# 쌍으로 존재하기 때문에 변수 하나 삭제

## 제거할 column 저장할 리스트
remove = []
seen = []
pair = []

for key, value in above.items():
    # seen 리스트에 한 번 검사한 것들 저장
    seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            if x not in seen:
                # 쌍 중에 하나만 제거
                remove.append(x)
                pair.append(key)

remove = list(set(remove))

In [ ]:
train = train.drop(columns = remove)
test = test.drop(columns = remove)

In [ ]:
train.head()

# Modeling

In [ ]:
# ids 추출
train_ids = train['SK_ID_CURR']
test_ids = test['SK_ID_CURR']

# id와 target 값 제거 
features = train.drop(columns = ['SK_ID_CURR', 'TARGET'])
test_features = test.drop(columns=['SK_ID_CURR'])

# 훈련 단계에 사용되는 label 추출
labels = train['TARGET']

### Encoding
train 데이터에 string이 있기 때문에 인코딩을 진행해야 함

In [ ]:
# one-hot 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# train과 test에 들어간 column 일치시키기
train, test = train.align(test, join='inner', axis=1)

In [ ]:
train.head()

In [ ]:
# features 이름 추출
feature_names = list(train.columns)

In [ ]:
#np array로 변환
train = np.array(train)
test = np.array(test)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(train, labels, test_size=0.3, random_state=2020)

In [ ]:
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
preds = clf.predict_proba(test, axis=1)[:, 1 ]

In [ ]:
submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': preds})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('BigData_EDA_FE_SHUN.csv', index=False)